# Storm Data Classification

In [27]:
import pandas as pd
import numpy as np
import os

from numpy import array
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [3]:
# df = pd.read_csv(r'/Users/danielschneider/Downloads/mi_2015_d.csv')
# df.head(50)
# # df.dropna(axis=1,inplace=True)

# mi = df[df['STATE'] == 'MICHIGAN']
# mi
# mi['BEGIN_YEARMONTH'] = mi['BEGIN_YEARMONTH'].apply(str)
# mi['Year'] = mi['BEGIN_YEARMONTH'].str[:4]
# mi['Month'] = mi['BEGIN_YEARMONTH'].str[5:]
# mi = mi.rename(columns={'BEGIN_DAY':'Day'})
# mi.drop(columns=['STATE_FIPS','DATA_SOURCE','MONTH_NAME'],inplace=True)
# mi.columns
# mi.to_csv(r'/Users/danielschneider/Desktop/m1-2021/m1-2021/mi_2015_damage.csv')
# mi

path = r'C:/Users/Alex Dean/Desktop/EcoData 2022/Actual Project/m1-2021'
os.chdir(path)
directory = os.getcwd() + "/"

base = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        current_csv = pd.read_csv(filename)
        merge = [base, current_csv]
        base = pd.concat(merge,ignore_index=True)
        
base['BEGIN_YEARMONTH'].sort_values()

base.to_csv(r'C:/Users/Alex Dean/Desktop/EcoData 2022/Actual Project/m1-2021/merged_years.csv')

C:\Users\ALEXDE~1\AppData\Local\Temp/ipykernel_14468/1436305437.py:24: DtypeWarning: Columns (31,36,37,39,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  current_csv = pd.read_csv(filename)


In [4]:
base
base.sort_values('BEGIN_YEARMONTH',inplace=True)
base.reset_index(inplace=True)
base.drop(columns=['Unnamed: 0', 'index'],inplace=True)
base.columns

Index(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'BEGIN_YEARMONTH',
       'Day', 'BEGIN_TIME', 'END_YEARMONTH', 'END_DAY', 'END_TIME',
       'EPISODE_ID', 'EVENT_ID', 'STATE', 'YEAR', 'EVENT_TYPE', 'CZ_TYPE',
       'CZ_FIPS', 'CZ_NAME', 'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE',
       'END_DATE_TIME', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
       'SOURCE', 'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY',
       'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO',
       'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME',
       'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE',
       'END_AZIMUTH', 'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT',
       'END_LON', 'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'Year', 'Month',
       'BEGIN_DAY', 'STATE_FIPS', 'MONTH_NAME', 'DATA_SOURCE'],
      dtype='object')

In [5]:
base

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,BEGIN_YEARMONTH,Day,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,...,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,Year,Month,BEGIN_DAY,STATE_FIPS,MONTH_NAME,DATA_SOURCE
0,NaN,NaN,NaN,199001,25.0,1238,199001,25,1238,NaN,...,NaN,NaN,NaN,NaN,1990.0,1.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,199001,25.0,1215,199001,25,1215,NaN,...,NaN,NaN,NaN,NaN,1990.0,1.0,NaN,NaN,NaN,NaN
2,26437.0,26437.0,NaN,199001,25.0,1221,199001,25,1221,NaN,...,NaN,NaN,NaN,NaN,1990.0,1.0,NaN,NaN,NaN,NaN
3,26436.0,26436.0,NaN,199001,25.0,1220,199001,25,1220,NaN,...,NaN,NaN,NaN,NaN,1990.0,1.0,NaN,NaN,NaN,NaN
4,26435.0,26435.0,NaN,199001,25.0,1215,199001,25,1215,NaN,...,NaN,NaN,NaN,NaN,1990.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156670,NaN,NaN,NaN,202012,12.0,300,202012,12,1900,155039.0,...,NaN,NaN,Low pressure crossed southern lower Michigan d...,NaN,2020.0,2.0,NaN,NaN,NaN,NaN
156671,NaN,NaN,NaN,202012,12.0,300,202012,12,1900,155039.0,...,NaN,NaN,Low pressure crossed southern lower Michigan d...,NaN,2020.0,2.0,NaN,NaN,NaN,NaN
156672,NaN,NaN,NaN,202012,12.0,300,202012,12,1700,153958.0,...,NaN,NaN,A winter storm brought 6 to over 8 inches of s...,Six to eight inches of snow fell and there wer...,2020.0,2.0,NaN,NaN,NaN,NaN
156673,69672.0,69672.0,NaN,202012,23.0,1600,202012,25,1100,154566.0,...,NaN,NaN,Arctic air pouring across Lake Superior behind...,The observer at Painesdale measured a two-day ...,2020.0,2.0,NaN,NaN,NaN,NaN


In [6]:
select = base[['BEGIN_YEARMONTH', 'Day', 'BEGIN_TIME', 'END_TIME', 'EVENT_TYPE', 'EVENT_ID','STATE', 'YEAR', 'DAMAGE_PROPERTY', 'MAGNITUDE','Month', 'CZ_FIPS','CZ_NAME','TOR_F_SCALE', 'FLOOD_CAUSE']]
select['DAMAGE_PROPERTY'].unique()
select = select.fillna('0')
select
select['DAMAGE_PROPERTY'] = select['DAMAGE_PROPERTY'].astype(str)
select['DAMAGE_PROPERTY'] = select['DAMAGE_PROPERTY'].map(lambda x: x.rstrip('K'))
# select[select['DAMAGE_PROPERTY'].str.match('[\w\d\.][M$]')].unique()



In [7]:
select

,BEGIN_YEARMONTH,Day,BEGIN_TIME,END_TIME,EVENT_TYPE,EVENT_ID,STATE,YEAR,DAMAGE_PROPERTY,MAGNITUDE,Month,CZ_FIPS,CZ_NAME,TOR_F_SCALE,FLOOD_CAUSE
0,199001,25.0,1238,1238,Thunderstorm Wind,10054097,MICHIGAN,1990,0,80.0,1.0,161,0,0,0
1,199001,25.0,1215,1215,Thunderstorm Wind,10054091,MICHIGAN,1990,0,0.0,1.0,163,0,0,0
2,199001,25.0,1221,1221,Thunderstorm Wind,10054094,MICHIGAN,1990,0,0.0,1.0,161,0,0,0
3,199001,25.0,1220,1220,Thunderstorm Wind,10054093,MICHIGAN,1990,0,0.0,1.0,115,0,0,0
4,199001,25.0,1215,1215,Thunderstorm Wind,10054092,MICHIGAN,1990,0,56.0,1.0,163,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156670,202012,12.0,300,1900,Winter Storm,935258,MICHIGAN,2020,0.00,0,2.0,34,ROSCOMMON,0,0
156671,202012,12.0,300,1900,Winter Storm,935260,MICHIGAN,2020,0.00,0,2.0,35,OGEMAW,0,0
156672,202012,12.0,300,1700,Winter Storm,927140,MICHIGAN,2020,50.00,0,2.0,39,OSCEOLA,0,0
156673,202012,23.0,1600,1100,Winter Weather,931205,MICHIGAN,2020,0.00,0,2.0,3,NORTHERN HOUGHTON,0,0


In [8]:
select.CZ_NAME.sort_values()

0              0
1823           0
1824           0
1825           0
1826           0
          ...   
20225    ZEIBACH
28425    ZEIBACH
38926    ZIEBACH
33543    ZIEBACH
39626    ZIEBACH
Name: CZ_NAME, Length: 156675, dtype: object

In [9]:
select['CZ_FIPS'].unique()

array([161, 163, 115,  91, 155, 125,  99,  55,   5, 159,  59, 123, 117,
       165,  75,  77,  25,  31,  27,  65,  45,  37,   9,  49,  81, 121,
       139, 153, 137,   7, 119,   1, 135,  39,  11,  69, 147,  33,  23,
       111, 151,  93,  67, 145,  57,  87,  15,  17,  73,  85, 127, 157,
        63, 113, 143,  51, 149, 129, 107, 133,  21,  35,  19, 101, 109,
       141,  29,  95,   3,  61,  83,  71,  47, 105,  41, 103, 365, 315,
        97, 423,  89,  13, 401, 131, 309, 455, 223, 499, 379, 203, 343,
       191, 285, 471, 459, 417, 485,  79, 253, 503, 447, 297, 167, 259,
       363, 187, 209, 189,   0, 197, 177, 169, 245, 349, 479, 215,  43,
       217, 183, 325, 487, 173, 185, 201, 219, 181, 497, 257, 355, 175,
       481,  53, 439, 229, 329, 179, 291, 443, 231, 347, 213, 225, 469,
       397, 171, 233, 303, 193, 373, 199, 211, 441, 205, 207, 295, 251,
       249, 221, 277, 367, 337, 465, 313, 457, 467, 387, 269, 453, 381,
       473, 419, 357, 237, 409, 415, 429, 375, 393, 339, 371, 27

In [10]:
select.DAMAGE_PROPERTY = (select.DAMAGE_PROPERTY.replace(r'[hKMB]+$','',regex=True).astype(float) * \
                         select.DAMAGE_PROPERTY.str.extract(r'[\d\.]+([hKMB]+)',expand=False).fillna(1)
                                                        .replace(['h','K','M','B'],[100,10**3,10**6,10**9]).astype(int))

In [11]:
select.DAMAGE_PROPERTY.sort_values()

0         0.000000e+00
98313     0.000000e+00
98312     0.000000e+00
98311     0.000000e+00
98310     0.000000e+00
              ...     
135764    4.000000e+08
135689    1.100000e+09
135508    1.100000e+09
135513    1.100000e+09
135626    1.100000e+09
Name: DAMAGE_PROPERTY, Length: 156675, dtype: float64

In [12]:
select.CZ_FIPS

0         161
1         163
2         161
3         115
4         163
         ... 
156670     34
156671     35
156672     39
156673      3
156674     28
Name: CZ_FIPS, Length: 156675, dtype: int64

In [13]:
df = pd.read_csv(
    'population.txt',sep='\t',header=None)

In [14]:
df.drop(columns=[0,3], inplace=True)

In [15]:
df['CZ_FIPS'] = df[2] - 26000

In [16]:
df['YEAR'] = df[4]

In [17]:
merged = df.merge(select, how='inner', on=['CZ_FIPS', 'YEAR'])

In [18]:
merged.tail(50)
merged.drop_duplicates(inplace=True)

In [19]:
merged.head(1200)

,1,2,4,5,CZ_FIPS,YEAR,BEGIN_YEARMONTH,Day,BEGIN_TIME,END_TIME,EVENT_TYPE,EVENT_ID,STATE,DAMAGE_PROPERTY,MAGNITUDE,Month,CZ_NAME,TOR_F_SCALE,FLOOD_CAUSE
0,"Alcona County, MI",26001,1990,10169,1,1990,199008,27.0,1855,1855,Thunderstorm Wind,10055210,MICHIGAN,0.0,64.0,8.0,0,0,0
3,"Alcona County, MI",26001,1990,10169,1,1990,199010,4.0,1300,1300,Thunderstorm Wind,10054154,MICHIGAN,0.0,0.0,0.0,0,0,0
6,"Alcona County, MI",26001,1991,10312,1,1991,199103,27.0,1840,1840,Tornado,10055293,MICHIGAN,2500000.0,0.0,3.0,0,0,0
7,"Alcona County, MI",26001,1991,10312,1,1991,199103,27.0,1835,1835,Tornado,10055291,MICHIGAN,2500000.0,0.0,3.0,0,0,0
8,"Alcona County, MI",26001,1991,10312,1,1991,199103,27.0,1905,1905,Thunderstorm Wind,10055307,MICHIGAN,0.0,62.0,3.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4364,"Alger County, MI",26003,2014,9370,3,2014,201402,17.0,1200,0,Winter Weather,501696,MICHIGAN,0.0,0,2.0,NORTHERN HOUGHTON,0,0
4378,"Alger County, MI",26003,2014,9370,3,2014,201403,21.0,100,700,Winter Weather,507946,MICHIGAN,0.0,0,3.0,NORTHERN HOUGHTON,0,0
4379,"Alger County, MI",26003,2014,9370,3,2014,201403,27.0,700,700,Winter Weather,507981,MICHIGAN,0.0,0,3.0,NORTHERN HOUGHTON,0,0
4380,"Alger County, MI",26003,2014,9370,3,2014,201403,21.0,1300,1900,Winter Weather,507939,MICHIGAN,0.0,0,3.0,NORTHERN HOUGHTON,0,0


In [20]:
merged = merged.loc[merged['STATE'] == 'MICHIGAN']

In [21]:
merged

,1,2,4,5,CZ_FIPS,YEAR,BEGIN_YEARMONTH,Day,BEGIN_TIME,END_TIME,EVENT_TYPE,EVENT_ID,STATE,DAMAGE_PROPERTY,MAGNITUDE,Month,CZ_NAME,TOR_F_SCALE,FLOOD_CAUSE
0,"Alcona County, MI",26001,1990,10169,1,1990,199008,27.0,1855,1855,Thunderstorm Wind,10055210,MICHIGAN,0.0,64.0,8.0,0,0,0
3,"Alcona County, MI",26001,1990,10169,1,1990,199010,4.0,1300,1300,Thunderstorm Wind,10054154,MICHIGAN,0.0,0.0,0.0,0,0,0
6,"Alcona County, MI",26001,1991,10312,1,1991,199103,27.0,1840,1840,Tornado,10055293,MICHIGAN,2500000.0,0.0,3.0,0,0,0
7,"Alcona County, MI",26001,1991,10312,1,1991,199103,27.0,1835,1835,Tornado,10055291,MICHIGAN,2500000.0,0.0,3.0,0,0,0
8,"Alcona County, MI",26001,1991,10312,1,1991,199103,27.0,1905,1905,Thunderstorm Wind,10055307,MICHIGAN,0.0,62.0,3.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125630,"Wexford County, MI",26165,2018,33452,165,2018,201808,28.0,1833,1840,Thunderstorm Wind,782805,MICHIGAN,0.0,0,8.0,WEXFORD,0,0
125637,"Wexford County, MI",26165,2019,33654,165,2019,201907,19.0,2355,2355,Thunderstorm Wind,848256,MICHIGAN,0.0,0,7.0,WEXFORD,0,0
125641,"Wexford County, MI",26165,2020,33743,165,2020,202004,7.0,1855,1903,Hail,889996,MICHIGAN,0.0,1.25,4.0,WEXFORD,0,0
125642,"Wexford County, MI",26165,2020,33743,165,2020,202004,7.0,1757,1757,Hail,889995,MICHIGAN,0.0,0.75,4.0,WEXFORD,0,0


In [22]:
pops = df
pops['county'] = pops.loc[:,1].str.split(' ').str[0]
pops['county'] = pops['county'].str.upper()
mi_counties = list(pops['county'].unique())
merged = merged[merged.CZ_NAME.isin(mi_counties)]
merged

,1,2,4,5,CZ_FIPS,YEAR,BEGIN_YEARMONTH,Day,BEGIN_TIME,END_TIME,EVENT_TYPE,EVENT_ID,STATE,DAMAGE_PROPERTY,MAGNITUDE,Month,CZ_NAME,TOR_F_SCALE,FLOOD_CAUSE
36,"Alcona County, MI",26001,1993,10316,1,1993,199308,30.0,2215,2215,Thunderstorm Wind,10329485,MICHIGAN,0.0,0.0,8.0,ALCONA,0,0
142,"Alcona County, MI",26001,1994,10560,1,1994,199405,0,1655,1655,Hail,10329486,MICHIGAN,0.0,0.75,0,ALCONA,0,0
429,"Alcona County, MI",26001,1994,10560,1,1994,199407,0,1444,1444,Thunderstorm Wind,10329487,MICHIGAN,0.0,0.0,0,ALCONA,0,0
430,"Alcona County, MI",26001,1994,10560,1,1994,199407,0,1511,1511,Thunderstorm Wind,10329488,MICHIGAN,0.0,0.0,0,ALCONA,0,0
431,"Alcona County, MI",26001,1994,10560,1,1994,199407,0,1538,1538,Hail,10329489,MICHIGAN,0.0,0.75,0,ALCONA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125630,"Wexford County, MI",26165,2018,33452,165,2018,201808,28.0,1833,1840,Thunderstorm Wind,782805,MICHIGAN,0.0,0,8.0,WEXFORD,0,0
125637,"Wexford County, MI",26165,2019,33654,165,2019,201907,19.0,2355,2355,Thunderstorm Wind,848256,MICHIGAN,0.0,0,7.0,WEXFORD,0,0
125641,"Wexford County, MI",26165,2020,33743,165,2020,202004,7.0,1855,1903,Hail,889996,MICHIGAN,0.0,1.25,4.0,WEXFORD,0,0
125642,"Wexford County, MI",26165,2020,33743,165,2020,202004,7.0,1757,1757,Hail,889995,MICHIGAN,0.0,0.75,4.0,WEXFORD,0,0


In [26]:
#loop for every year starting at 1990?
#do boolean logic similar to below for county and year to merge in population
#

tornado_bool = merged['EVENT_TYPE'] == 'Tornado'
tornado_merged = merged[tornado_bool]
tornado_merged

hail_bool = merged['EVENT_TYPE'] == 'Hail'
hail_merged = merged[hail_bool]
hail_merged


,1,2,4,5,CZ_FIPS,YEAR,BEGIN_YEARMONTH,Day,BEGIN_TIME,END_TIME,EVENT_TYPE,EVENT_ID,STATE,DAMAGE_PROPERTY,MAGNITUDE,Month,CZ_NAME,TOR_F_SCALE,FLOOD_CAUSE
142,"Alcona County, MI",26001,1994,10560,1,1994,199405,0,1655,1655,Hail,10329486,MICHIGAN,0.0,0.75,0,ALCONA,0,0
431,"Alcona County, MI",26001,1994,10560,1,1994,199407,0,1538,1538,Hail,10329489,MICHIGAN,0.0,0.75,0,ALCONA,0,0
523,"Alcona County, MI",26001,1994,10560,1,1994,199407,0,1610,1610,Hail,10329490,MICHIGAN,0.0,0.75,0,ALCONA,0,0
654,"Alcona County, MI",26001,1995,10775,1,1995,199505,28.0,1540,1540,Hail,10329491,MICHIGAN,0.0,0.75,5.0,ALCONA,0,0
659,"Alcona County, MI",26001,1995,10775,1,1995,199507,14.0,1735,1735,Hail,10329494,MICHIGAN,0.0,1.0,7.0,ALCONA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125565,"Wexford County, MI",26165,2011,32637,165,2011,201105,29.0,1916,1916,Hail,315666,MICHIGAN,0.0,0.88,5.0,WEXFORD,0,0
125580,"Wexford County, MI",26165,2012,32518,165,2012,201207,25.0,1954,2003,Hail,399956,MICHIGAN,0.0,2.0,7.0,WEXFORD,0,0
125641,"Wexford County, MI",26165,2020,33743,165,2020,202004,7.0,1855,1903,Hail,889996,MICHIGAN,0.0,1.25,4.0,WEXFORD,0,0
125642,"Wexford County, MI",26165,2020,33743,165,2020,202004,7.0,1757,1757,Hail,889995,MICHIGAN,0.0,0.75,4.0,WEXFORD,0,0
